In [ ]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import cv2
import numpy as np
import time
import mediapipe 
from cvzone.FPS import FPS
import os
import math
import csv
import time
############# our Variables ################################################################################################
#wCam, hCam = 640, 480
wCam, hCam = 1280, 720
cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
finalText=''
buttonList=[]
keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

#######################################detector#############################################################################
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)   
############################################################################################################################
class Button():
    def __init__(self,pos,text,size=[85,85]) :
        self.pos = pos
        self.text = text
        self.size = size

def calculateHandArea(hand):
    xhand, yhand, handWidth, handHeight = hand['bbox']
    area = handWidth * handHeight
    if area >int(35000):
        cv2.putText(frame,"Please Keep Your Hand Away",(50,50),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)

    return area


def drawAllRectangles(image,buttonList):
    #get all buttons
    for button in buttonList :
        x,y =button.pos
        w,h = button.size
        cv2.rectangle(image,(button.pos[0],button.pos[1]),(x+w,h+y),(0,255,0),cv2.FILLED)
        #cv2.putText(img,self.text,(65,83),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
        cv2.putText(image,button.text,(x+30,y+60),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
    return image
    
#add all buttons opject in button list    
for i in range(len(keys)):
    for j,key  in enumerate(keys[i]) :
        buttonList.append(Button([100*j +50,90*i+300],key))

while True :
    _,frame = cap.read()
    frame = cv2.flip(frame,1)
   # cv2.rectangle(frame,(50,50),(100,100),(0,255,0),5)
   # cv2.putText(frame,'A',(65,83),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
   #buttonList.append(Button([70*x+300,200],'A'))
    hands , _ = detector.findHands(frame,draw=False,flipType=True)
    frame = drawAllRectangles(frame,buttonList)

    if hands:
        hand = hands[0]
        #print(handHeight,handWidth)
        handList = hand['lmList']
        #start detect key
        for button in buttonList :
            x,y = button.pos
            w,h = button.size
            if x<handList[8][0]<x+w and y<handList[8][1]<y+h:
                print(x,y)
                print(handList[8][0],handList[8][1])
                cv2.rectangle(frame,(button.pos),(x+w,h+y),(0,255,255),cv2.FILLED)
                cv2.putText(frame,button.text,(x+30,y+60),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                length ,info ,_ = detector.findDistance(handList[4][0:2],handList[8][0:2])
                handArea = calculateHandArea(hand)
               # print(length)
                if length <30 :
                    cv2.rectangle(frame,(button.pos),(x+w,h+y),(255,0,0),cv2.FILLED)
                    cv2.putText(frame,button.text,(x+30,y+60),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                    finalText += button.text
                    time.sleep(0.15)

    cv2.rectangle(frame, (50, 50), (500, 150), (255, 0, 0), cv2.FILLED)
    cv2.putText(frame, finalText, (100, 130),
                cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)


    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[50, 566])
    #cv2.imshow('MVIRTUALin Image ',frame)
    cv2.imshow('Ai Vertuial Key Board  ',frame )
    # Display the camera feed
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows() 
